### ***Insights into academia***

* How has the open-access publishing grown over the years? 
  * Does it improve the quality of research? How well does open-access translate to the quantity of research? 
  * Can we devise a normalized research quality metric based on the information?
  * To what extend does the open-access benefit the community, in particular, the developing nations? 

In an attempt to make federally funded research open, initiated by scientists and researchers, Library Genesis is a database for articles and books on various topics, which allows free access to content that is otherwise paywalled or not digitized elsewhere. 

> A sister site, Sci-Hub's, founder *Alexandra Elbakyan* was listed by *Nature*, in 2016, among the **top ten people that mattered in science**. US ranks *second* in the terms of web-traffic on Sci-Hub and related websites. 
>
>As per wikipedia: Elbakyan's Sci-Hub is widely used in both developed and developing countries, serving over 200,000 requests per day as of February 2016. How has the inception of Sci-Hub affected the quality/quantity of research in developing nations? Do more authors go on to write high quality papers in their respective fields? Has the number of papers cited per paper increased (with access to large number of papers and therefore more information)? Does their sphere of influence increase? Do they get more post-doctoral offers abroad? (This can be partially answered by looking at the collaboration graph of a researcher)

In this work, we shall restrict our attention to academic papers. Our study shall be based on ***database record*** of academic papers, ***not*** the actual academic papers themselves. The database, publicly available at http://booksdescr.org/dbdumps/scimag, contains over 70 million records of academic papers. It includes information of paper titles, author list, doi, etc. which in turn can be used to query citation information, abstract, etc. from Google Scholar, Pubmed, Researchgate, etc. The important point is: ***the database is relatively unanalyzed, and LibGen is perhaps the biggest such record of academic papers!*** The databse is incredibly information rich, and can be utilized to reveal insights into publishing. 

* How does the paper quality of a researcher changes over time in various fields? For example: It is widely believed in Mathematics that ground-breaking contribution are from researchers below 30-35 years of age. How does that vary in other fields, by country of origin, by institution, etc? The intention of this study shall be to provide answers backed by analysis on actual data.

> Important Note:
>* Our study shall be based on ***database record*** of academic papers, ***not*** the actual academic papers themselves. LibGen has been the subject of various formal academic studies. Few of them are listed below:
>  * Karaganis, Joe. *Shadow libraries: access to knowledge in global higher education*. MIT Press, 2018.
>  * Cabanac, Guillaume. *Bibliogifts in LibGen? A study of a text‐sharing platform driven by biblioleaks and crowdsourcing.* Journal of the Association for Information Science and Technology 67, no. 4 (2016): 874-884.   

Data presentation: 
* The intention will be to generate an interactive insightful tool on academic publishing (see for example: Hans Rosling's https://www.gapminder.org/tools on socio-economic growth across the globe over years).

Tools:
* Python: Apache-Spark's ML on large datasets, MySQL, NetworkX, etc; Javascript: d3js

In [1]:
import numpy as np
import itertools as it
from collections import defaultdict
import pandas as pd
pd.set_option('display.max_colwidth', -1)

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import pyspark
import MySQLdb
import requests

In [2]:
# Connect to LibGen's publicly available database
db=MySQLdb.connect(user="jaiswal0", passwd="dInc#2019", db="8k")

In [3]:
# We shall analyze a subportion (~10.8 million) of the entire database (>70 million).
num_entries = pd.read_sql('SELECT COUNT(*) FROM scimag', con=db)
print(num_entries)

   COUNT(*)
0  10795980


In [4]:
# What are the columns
cols = pd.read_sql('''SELECT column_name FROM information_schema.columns where table_name='scimag' AND table_schema='2k' ''', con=db)
print(cols['COLUMN_NAME'].to_string(index=False).replace("\n", ""))

 AbstractURL Attribute1  Attribute2  Attribute3  Attribute4  Attribute5  Attribute6  Author      Day         DOI         DOI2        Filesize    First_page  ID          ISBN        ISSNE       ISSNP       Issue       Journal     JOURNALID   Last_page   MD5         Month       PII         PMC         PubmedID    TimeAdded   Title       visible     Volume      Year       


In [5]:
# Find entries with a PubmedID
art_pubmed = pd.read_sql('''
    SELECT Author, PubmedID, Year, Attribute1 FROM scimag 
    where PubmedID>0 
''', con=db)
print(art_pubmed[:10])

                                                                                                                                                                                                                      Author  \
0  Katherine M. Hegmann; Aimee S. Spikes; Avi Orr-Urtreger; Lisa G. Shaffer                                                                                                                                                    
1  Vijay Tonk; Nancy R. Schneider; Mauricio R. Delgado; Jen-i Mao; Roger A. Schultz                                                                                                                                            
2  Selma Siegel Witchel; Peter A. Lee; Massimo Trucco                                                                                                                                                                          
3  Orit Reish; Susan A. Berry; Gordon Dewald; Richard A. King                                           

In [6]:
# Number of entries with a pubmedID
len(art_pubmed)

1616069

### ^^ Nearly 1.6 million entries have a *pubmedID*. Having identified this information, we can query abstract information from NIH database

In [ ]:
url = '''https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id={id}'''
headers = {'User-Agent': 'Hydra/1.3.15'}

def load_medline(info):
    data = info.split("\n");
    infodict = defaultdict(lambda: None)
    temp = ['', '']
    for line in data:
        if not line: continue
        if line[4]=="-":
            if temp[0]!='':
              if temp[0] in infodict:
                infodict[temp[0]][-1] = temp[1];            
              else:
                infodict[temp[0]] = [temp[1]]; 
            temp = [line[:4].strip(), line[5:]]
            if temp[0] in infodict:
              infodict[temp[0]].append(temp[1]);
            else:
              infodict[temp[0]] = [temp[1]];
        else:
            temp[1] += ' ' + line.strip() + ' '    
            continue
    return dict({"Abstract": infodict["AB"]})

def fetch_pubmed(pubmedID):
    api_url = url.format(id=pubmedID)
    #print(api_url)
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return load_medline(response.content.decode('utf-8'))
    else:
        return None

# for all the entries fetch keywords
for entry in range(len(art_pubmed)): 
    pubmedID = art_pubmed['PubmedID'][entry]
    info = fetch_pubmed(pubmedID)
    #print(info)
    if info:
        #print(info["Abstract"], art_pubmed['Year'][entry])
        art_pubmed['Attribute1'][entry] = info["Abstract"]


https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741910
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741911
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741909
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741912
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741913
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741914
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741915
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741916
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741917
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741918
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741919
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741921
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8741922
https://api.ncbi.nlm.nih.

https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882391
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882392
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882393
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882394
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882395
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882396
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882397
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882398
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882399
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882400
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882401
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882402
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8882403
https://api.ncbi.nlm.nih.

https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723086
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723087
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723088
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723089
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723079
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723090
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723091
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723092
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723093
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8723094
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8725780
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8725781
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8725782
https://api.ncbi.nlm.nih.

https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837700
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837701
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837702
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837703
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837704
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837705
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837706
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837707
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837708
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837709
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837710
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837711
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8837712
https://api.ncbi.nlm.nih.

https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911599
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911600
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911601
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911602
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911603
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911604
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911605
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911606
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911607
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911608
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911609
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911610
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8911611
https://api.ncbi.nlm.nih.

...

https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630923
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630924
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630925
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630926
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630927
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630928
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630929
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630930
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630931
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630932
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630933
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630934
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8630935
https://api.ncbi.nlm.nih.

https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608533
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608534
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608535
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608536
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608537
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608538
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608539
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608540
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608541
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608542
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608543
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608544
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8608545
https://api.ncbi.nlm.nih.

https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635091
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635092
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635093
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635094
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635095
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635096
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635097
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635098
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635099
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635100
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635101
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635102
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8635103
https://api.ncbi.nlm.nih.

https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8674019
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8674020
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8674021
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8674022
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8697380
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8697381
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8697382
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8697383
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8697384
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8697385
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8697386
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8697387
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8697388
https://api.ncbi.nlm.nih.

https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826937
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826938
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826939
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826940
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826941
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826942
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826943
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826944
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826945
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826946
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826947
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826948
https://api.ncbi.nlm.nih.gov/lit/ctxp/v1/pubmed/?format=medline&id=8826949
https://api.ncbi.nlm.nih.

In [ ]:
# Find entries for Nature Scientific reports
art_scirep = pd.read_sql('''
    SELECT Title FROM scimag 
    where Journal like '%American%' 
''', con=db)

In [ ]:
len(art_scirep)